<h1>1. Business Problem</h1>

<h2>1.1. Description</h2>
<p>Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion (only for manager position and below) and prepare them in time. </p>
<p>Currently the process, they are following is:
They first identify a set of employees based on recommendations/ past performance
Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
<p>
At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion
For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle.
<p>
They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.
<p>
  
<h2>1.2. Source/Useful Links</h2>
<p>https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018/</p>
<p>https://tech.yandex.com/catboost/</p>
<h1>2. Machine Learning Problem Formulation</h1>
<h2>2.1. Data</h2>

<h3>2.1.1. Data Overview</h3>

- Source: https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018/
<p>WNS Analytics Wizard 2018 Data hack competition from analyticsvidhya.com, same data set i am using</p>
<p>Training Data: it have 54808 records and 14 columns</p>
<p>Test Data: it have 23490 records and 13 columns</p>
 
<h2>2.2. Mapping the real-world problem to an DL problem</h2>
<h3>2.2.1. Type of Deep Learning Problem</h3>

<p>Binary Classification :</p>
- Based on Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.:

<h3>2.2.2. Performance Metric</h3>

Metric(s): F1 score 
<p>
 data set have unbalanced data[9:1] so better to select F1 score instead of Accuracy
</p>
<h2>
2.3. Train, CV and Test Datasets
</h2>
Split the Training  dataset  into Two parts train, and cross validation with 70% and 30% of data respectively

<h1>Steps to Mount Google Drive with Google Colab </h1>
Going to Use Google Colab so first need to put data-set into one folder in google Drive.
Need to mount google drive with colab.
below stpes need to uncomment for first time 


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
#Data set is present in path :/content/drive/Hackthon/WNS/ of google drivw
!cp  /content/drive/Hackthon/WNS/* .
!ls -l

total 6696
-rw-r--r-- 1 root root    2495 Sep 15 20:29 adc.json
drwxr-xr-x 2 root root    4096 Sep 15 20:30 drive
drwxr-xr-x 2 root root    4096 Sep 13 17:28 sample_data
-rw-r--r-- 1 root root 1565147 Sep 15 20:30 test_2umaH9m.csv
-rw-r--r-- 1 root root 3759647 Sep 15 20:30 train_LZdllcl.csv
-rw-r--r-- 1 root root     229 Sep 15 20:30 Untitled
-rw-r--r-- 1 root root  202021 Sep 15 20:30 WNS_Traine_01.ipynb
-rw-r--r-- 1 root root 1178424 Sep 15 20:30 WNS_Traine_02.ipynb
-rw-r--r-- 1 root root   57567 Sep 15 20:30 WNS_Traine_03.ipynb
-rw-r--r-- 1 root root   62664 Sep 15 20:30 WNS_Traine_04.ipynb


In [0]:
# Installing caboost model before use
!pip install catboost
!pip install ipywidgets
  


<h1>3. Exploratory Data Analysis</h1>
<h3>3.1. Import All required Classed first</h3>


In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error,f1_score,recall_score,precision_score
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from numpy import argmax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier

In [0]:
SEED = 0

<h1>3 Data preprocessing</h1>

<h2>3.1. Reading Data</h2>

In [0]:
Data = pd.read_csv("train_LZdllcl.csv")

In [0]:
print(Data.shape)
print(Data.columns)

(54808, 14)
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')


<h4>Note: </h4>Data set have total 14 feature "is_promoted" column is target feature and rest is input feature except "employee_id"

In [0]:
print(Data.describe())

        employee_id  no_of_trainings           age  previous_year_rating  \
count  54808.000000     54808.000000  54808.000000          50684.000000   
mean   39195.830627         1.253011     34.803915              3.329256   
std    22586.581449         0.609264      7.660169              1.259993   
min        1.000000         1.000000     20.000000              1.000000   
25%    19669.750000         1.000000     29.000000              3.000000   
50%    39225.500000         1.000000     33.000000              3.000000   
75%    58730.500000         1.000000     39.000000              4.000000   
max    78298.000000        10.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  \
count       54808.000000   54808.000000  54808.000000        54808.000000   
mean            5.865512       0.351974      0.023172           63.386750   
std             4.265094       0.477590      0.150450           13.371559   
min    

<h2>3. 2 Filling NAN records</h2>
<p>below count can se for previous_year_rating and education is records count is not 54808 ie there are some missing values which need to fill</p>

In [0]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               54808
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    54808
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [0]:
values={"previous_year_rating":3,"education":"O"}
Data.fillna(value=values,inplace=True)

In [0]:
data_xs = Data.drop(['employee_id',"is_promoted"],axis=1)

In [0]:
categorical_features_indices = np.array([0,1,2,3,4,6,8,10,11])

<h3>3.3 Train Test Splite</h3>

In [0]:
x_train,x_test,y_train,y_test = train_test_split(data_xs,Data["is_promoted"], test_size = 0.3,random_state=20)

<h2>Model Evalution</h2>

In [0]:
def model_evalution(model):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(x_train)
  test_pre = model.predict(x_test)
  train_pro = model.predict_proba(x_train)
  test_pro = model.predict_proba(x_test)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train F1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train recall: {0} \t Test recall: {1}".format(recall_score(y_train, train_pre),recall_score(y_test,test_pre)))
  print("Train precision: {0} \t Test Precision: {1}".format(precision_score(y_train, train_pre),precision_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score
  
  

<h1>4. Model definition and Training </h1>
<h3>catBoost </h3>
<p>below trained model are best model i found using variuos hyperparameter tuning.</p>
<p>For hyperparameter tuning part you can see other attached ipynb file</p>

In [0]:
catboost_model = CatBoostClassifier(iterations=400,verbose=True,eval_metric="F1",learning_rate=0.2,class_weights=[1,3],depth=3,l2_leaf_reg=100,bagging_temperature=0.5)
catboost_model.fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test),use_best_model=False)
model_evalution(catboost_model)

0:	learn: 0.2017291	test: 0.2112933	best: 0.2112933 (0)	total: 130ms	remaining: 51.7s
1:	learn: 0.2001404	test: 0.2109327	best: 0.2112933 (0)	total: 259ms	remaining: 51.5s
2:	learn: 0.2001404	test: 0.2109327	best: 0.2112933 (0)	total: 368ms	remaining: 48.8s
3:	learn: 0.2921274	test: 0.2803738	best: 0.2803738 (3)	total: 499ms	remaining: 49.4s
4:	learn: 0.3486821	test: 0.3464773	best: 0.3464773 (4)	total: 614ms	remaining: 48.5s
5:	learn: 0.5207840	test: 0.5328418	best: 0.5328418 (5)	total: 703ms	remaining: 46.2s
6:	learn: 0.5263383	test: 0.5371639	best: 0.5371639 (6)	total: 793ms	remaining: 44.5s
7:	learn: 0.5167928	test: 0.5238095	best: 0.5371639 (6)	total: 881ms	remaining: 43.2s
8:	learn: 0.5254422	test: 0.5243020	best: 0.5371639 (6)	total: 959ms	remaining: 41.7s
9:	learn: 0.5204260	test: 0.5216940	best: 0.5371639 (6)	total: 1.03s	remaining: 40.2s
10:	learn: 0.5206377	test: 0.5215137	best: 0.5371639 (6)	total: 1.11s	remaining: 39.4s
11:	learn: 0.5209228	test: 0.5235385	best: 0.5371639 

In [0]:
model_evalution(catboost_model)

<h2>5.1  Obervation</h2>as Above can see F1 score for Train  : 0.5415202515258 	and  Test  : 0.5366894197952219
<p>Result is not quit good for forther improvmentcan try with  stacking or other model like LightGBM</p>

<h2>6. Improvement</h2>
for further  one can try LightGBM, XgBoost and stacking model to get better F1 score

<h2>7. Challenge faced</h2>
As its high Unbalanced data set, 90% records having Not promoted. to handle this i have used class_weights=[1,3] of catboost model, but this took very much time to tune this parameter.

<h2>8. Prediction for Test data <h2>

In [0]:
Data_test = pd.read_csv("test_2umaH9m.csv")

In [0]:
values={"previous_year_rating":3,"education":"O"}
Data_test.fillna(value=values,inplace=True)

In [0]:
data_test_xs = Data_test.drop(['employee_id'],axis=1)

In [0]:
y_test_pre = catboost_model.predict(data_test_xs)

In [0]:
Data_test["is_promoted"] = y_test_pre

In [0]:
Data_test.to_csv("WNS_catBoost_Test_w_1_3_400_lr_0_2_l2_100.csv",columns=["employee_id","is_promoted"],index=False)
#Data_test.to_csv("WNS.csv",index=False)

In [0]:
from google.colab import files
files.download("WNS_catBoost_Test_w_1_3_400_lr_0_2_l2_100.csv")
#files.download("WNS.csv")

In [0]:
!cp WNS_catBoost_Test_w_1_3_400_lr_0_2_l2_100.csv /content/drive/Hackthon/WNS/